In [2]:
import pandas as pd
import math
from datetime import time
import re

In [7]:
import os
folder_path = "C:/Users/srmut/rohith_workspace/rohith_workspace/ML Datasets/chess/RawFiles/"
file_paths = os.listdir(folder_path)

In [8]:
def process_games(games):
    games = [i.split("\n") for i in games]
    for symbol in ["[","]","\""]:
        games = [[j.replace(symbol,"") for j in i] for i in games]
    games = [[j.split(" ",1) for j in i] for i in games]
    game_headers,game_stats = [[j[0] for j in i] for i in games][0], [[j[1] for j in i] for i in games]
    return(game_headers,game_stats)

def process_moves(moves):
    timestamps = [re.findall(r'\{.*?\}', i) for i in moves]
    for symbol in ["\n","...","{[%","clk","]}"]:
        moves = [i.replace(symbol,"") for i in moves]
    moves = [i.split(" ")[:-1] for i in moves]
    moves_only = [[j for j in i if len(j) > 1 and j[1] != "." and not j.isnumeric() and not j[:2].isnumeric()] for i in moves]
    first_move = [i[0]+"-"+i[2] for i in moves_only]
    number_of_moves = [math.ceil(len(i)/2) for i in timestamps]
    def calculate_time(time_var):
        m,s = list(map(int,time_var.split(":")))
        if m == 10:
            return "00:00"
        m,s = 9-m,59-s
        return time(minute = m,second = s)
    white_time_taken = [calculate_time(re.findall(r'\ (.*?)\]', i[::2][-1]).__getitem__(0)[2:7]) for i in timestamps]
    black_time_taken = [calculate_time(re.findall(r'\ (.*?)\]', i[1::2][-1]).__getitem__(0)[2:7]) for i in timestamps]
    moves_headers = ["first_move","number_of_moves","white_time_taken","black_time_taken"]
    move_stats = []
    for i in range(len(first_move)):
        move_stats.append([first_move[i],number_of_moves[i],white_time_taken[i],black_time_taken[i]])
    return moves_headers, move_stats


def process_file(file_path):
    with open(file_path,"r") as f:
        data = f.read()
    games = data.split("\n\n")[::2]
    moves = data.split("\n\n")[1::2]
    game_headers, game_stats = process_games(games)
    move_headers, move_stats = process_moves(moves)
    headers = game_headers + move_headers
    stats = []
    for i in range(len(game_stats)):
        stats.append(game_stats[i] + move_stats[i])
    df = pd.DataFrame(stats)
    df.columns = headers
    return df

In [9]:
def game_result(status):
    if "chandurohitheee" in status:
        return "won"
    elif "draw" in status:
        return "draw"
    else:
        return "loss"


In [11]:
df = pd.DataFrame()
for file_path in file_paths:
    df = pd.concat([df,process_file(folder_path+file_path)])
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(by = ["Date","EndTime"],ascending=False)
df["GameResult"] = df["Termination"].apply(game_result)
df.to_csv("chess_stats.csv",index=False)

In [12]:
df = pd.read_csv("chess_stats.csv")
df.head(3)

,Event,Site,Date,Round,White,Black,Result,WhiteElo,BlackElo,TimeControl,EndTime,Termination,first_move,number_of_moves,white_time_taken,black_time_taken,GameResult
0,Live Chess,Chess.com,2023-01-28,-,22roee22,chandurohitheee,0-1,1128,1268,600,16:12:13 PST,chandurohitheee won by resignation,e4-e5,33,00:05:26,00:03:04,won
1,Live Chess,Chess.com,2023-01-28,-,PrinterGoBrrrr,chandurohitheee,0-1,1333,1262,600,15:58:41 PST,chandurohitheee won by resignation,d4-Nc6,19,00:02:13,00:01:44,won
2,Live Chess,Chess.com,2023-01-28,-,chandurohitheee,sergo190281,0-1,1252,1275,600,15:39:58 PST,sergo190281 won by resignation,e4-e5,20,00:03:32,00:03:34,loss
